In [25]:
import pandas as pd
import psycopg2 as postgres


In [26]:
BASE = 'tp_dba'
USER = 'axelmoriceau'
PSWD = ''

conn = postgres.connect(
    host='localhost',
    database=BASE,
    user=USER,
    password=PSWD,
    port='5432'
)

cur = conn.cursor()

In [27]:
cur.execute(open("sql/questions/reset.sql", "r").read())

In [28]:
# requête interactive
region = 'aquitaine' # mettre a jour le contenu ici pour changer de région (i.e: tagne -> Bretagne, compte -> Franche compté etc)

cur.execute("select d.libelle from departements d join regions r using(reg) where r.ncc LIKE UPPER(%(region)s) or r.libelle LIKE UPPER(%(region)s) order by d.libelle", {'region': f'%{region}%'})
conn.commit()
data = cur.fetchall()

print(f'Les départements qui correspondent a la région {region}:')
for dep in data:
    print(f'\t- {dep[0]}')


Les départements qui correspondent a la région aquitaine:
	- Charente
	- Charente-Maritime
	- Corrèze
	- Creuse
	- Deux-Sèvres
	- Dordogne
	- Gironde
	- Haute-Vienne
	- Landes
	- Lot-et-Garonne
	- Pyrénées-Atlantiques
	- Vienne


In [29]:
commune = 'bordeaux'  # le nom de la ville doit être exact, sans accent ni caractère spéciaux
indicateur = 'pop'  # parmis DECE, LOG, LOGVAC, NAIS, NPER, PMEN, POP, RP, RSECOCC, SUPERF

cur.execute("select annee, ncc, dep,valeur from communes c join statistiques s on s.codgeo = c.com where upper(s.indicateur) = upper(%(indicateur)s) and c.ncc = upper( %(commune)s)", 
            {'commune': commune, 'indicateur': indicateur})
conn.commit()
data = cur.fetchall()

if(len(data)):
    print(f"Information pour l'indicateur {indicateur} pour la commune {commune} ({data[0][2]})")
    for info in data:
        print(f'\t{info[0]}: {info[3]:,}')
else:
    print(f'aucune information pour la commune de {commune} et l\'indicateur {indicateur}')

Information pour l'indicateur pop pour la commune bordeaux (33 )
	1968: 266,662.0
	1975: 223,131.0
	1982: 208,159.0
	1990: 210,336.0
	1999: 215,363.0
	2008: 235,891.0
	2013: 243,626.0
	2018: 257,068.0


In [30]:
# Question 1
cur.execute(open("sql/questions/question_1/question_1#1.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Les 10 villes les plus peuplées de France en 2018:')
for ville in data:
    print(f'\t- {ville[0]}: {ville[1]:,} habitants')


Les 10 villes les plus peuplées de France en 2018:
	- Paris: 2,175,601 habitants
	- Marseille: 868,277 habitants
	- Lyon: 518,635 habitants
	- Toulouse: 486,828 habitants
	- Nice: 341,032 habitants
	- Nantes: 314,138 habitants
	- Montpellier: 290,053 habitants
	- Strasbourg: 284,677 habitants
	- Bordeaux: 257,068 habitants
	- Lille: 233,098 habitants


In [31]:
# Question 2
cur.execute(open("sql/questions/question_1/question_1#2.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Évolution de la population Française au fil des années:')
for population in data:
    population = [0 if v is None else v for v in population]
    print(f'\n\t- Année {population[0]}: {population[1]:,} habitants -> +{population[2]:,} habitants ({population[3]}%)')
    print(f'\t\t- Naissances: {population[4]:,} | Décès: {population[5]:,}')
    if(population[2]):
        print(f'\t\t- Augmentation: {population[2]:,} || Naissances - Décès: {population[4] - population[5]:,} || Différence: {population[2]-(population[4] - population[5]):,}')


Évolution de la population Française au fil des années:

	- Année 1968: 50,798,112 habitants -> +0 habitants (0%)
		- Naissances: 6,149,732 | Décès: 3,909,870

	- Année 1975: 53,764,064 habitants -> +2,965,952 habitants (5.516606780320774%)
		- Naissances: 5,532,080 | Décès: 3,911,267
		- Augmentation: 2,965,952 || Naissances - Décès: 1,620,813 || Différence: 1,345,139

	- Année 1982: 55,569,542 habitants -> +1,805,478 habitants (3.2490424340729676%)
		- Naissances: 6,363,297 | Décès: 4,375,209
		- Augmentation: 1,805,478 || Naissances - Décès: 1,988,088 || Différence: -182,610

	- Année 1990: 58,040,659 habitants -> +2,471,117 habitants (4.257561927406786%)
		- Naissances: 6,877,418 | Décès: 4,819,081
		- Augmentation: 2,471,117 || Naissances - Décès: 2,058,337 || Différence: 412,780

	- Année 1999: 60,149,901 habitants -> +2,109,242 habitants (3.506642513010952%)
		- Naissances: 7,211,899 | Décès: 4,833,385
		- Augmentation: 2,109,242 || Naissances - Décès: 2,378,514 || Différence: -

In [32]:
# Question 3
cur.execute(open("sql/questions/question_1/question_1#3.sql", "r").read())
conn.commit()
data = cur.fetchall()

print('Ville la plus peuplée de chaque région en 2018:')
for population in data:
    print(f'\t- Région {population[2]} -> {population[1]} ({population[0]}): {population[3]:,} habitants')


Ville la plus peuplée de chaque région en 2018:
	- Région Île-de-France -> Paris (75056): 2,175,601 habitants
	- Région Provence-Alpes-Côte d'Azur -> Marseille (13055): 868,277 habitants
	- Région Auvergne-Rhône-Alpes -> Lyon (69123): 518,635 habitants
	- Région Occitanie -> Toulouse (31555): 486,828 habitants
	- Région Pays de la Loire -> Nantes (44109): 314,138 habitants
	- Région Grand Est -> Strasbourg (67482): 284,677 habitants
	- Région Nouvelle-Aquitaine -> Bordeaux (33063): 257,068 habitants
	- Région Hauts-de-France -> Lille (59350): 233,098 habitants
	- Région Bretagne -> Rennes (35238): 217,728 habitants
	- Région Normandie -> Le Havre (76351): 169,733 habitants
	- Région Bourgogne-Franche-Comté -> Dijon (21231): 156,854 habitants
	- Région La Réunion -> Saint-Denis (97411): 150,535 habitants
	- Région Centre-Val de Loire -> Tours (37261): 136,463 habitants
	- Région Martinique -> Fort-de-France (97209): 78,126 habitants
	- Région Corse -> Ajaccio (2A004): 70,817 habitants
	

In [33]:
# Création des vues

cur.execute(open("sql/questions/question_2/question_2#vue_region.sql", "r").read())
cur.execute(open("sql/questions/question_2/question_2#vue_departements.sql", "r").read())
conn.commit()


In [34]:
# Ajout des colonnes population
cur.execute(open("sql/questions/question_3/alter_tables.sql", "r").read())
cur.execute("select ncc,population from regions order by ncc limit 5;")
conn.commit()

data = cur.fetchall()
for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: None
BOURGOGNE FRANCHE COMTE: None
BRETAGNE: None
CENTRE VAL DE LOIRE: None
CORSE: None


In [35]:

# Creation et appel de la procédure
cur.execute(open("sql/questions/question_3/procedure.sql", "r").read())
cur.execute("call proc_update_population()")
cur.execute("select ncc,population from regions order by ncc limit 5;")
conn.commit()

data = cur.fetchall()
for region in data:
    print(f'{region[0]}: {region[1]}')


AUVERGNE RHONE ALPES: 7994459
BOURGOGNE FRANCHE COMTE: 2807807
BRETAGNE: 3335414
CENTRE VAL DE LOIRE: 2572853
CORSE: 338554


In [36]:
conn.close()